# Building a Fraud Prediction Model with EvalML

In this demo, we will build an optimized fraud prediction model using EvalML. To optimize the pipeline, we will set up an objective function to minimize the percentage of total transaction value lost to fraud. At the end of this demo, we also show much better this resulting pipeline performs compared to one trained just on AUC.

In [23]:
import evalml
from evalml.objectives import FraudCost

## Configure Fraud Objective

To  optimize the pipelines toward the specific business needs of this model, we can set your own assumptions for the cost of fraud. These parameters are

* `retry_percentage` - what percentage of customers will retry a transaction if it is declined?
* `interchange_fee` - how much of each successfull transaction do you collect?
* `fraud_payout_percentage` - how percentage of fraud will you be unable to collect
* `amount_col` - the column in the data the represents the transaction amount

Using these parameters, EvalML determines attempt to build a pipeline that will minimize the financial loss due to fraud.

In [21]:
fraud_objective = FraudCost(
    retry_percentage=.5,
    interchange_fee=.02,
    fraud_payout_percentage=.75,
    amount_col='amount',  
)

## Search for best modeling pipeline

In order to validate the results of the pipeline creation and optimization process, we will save some of our data as a holdout set

In [19]:
X, y = evalml.demos.load_fraud()

             Number of Features
Boolean                       1
Categorical                   6
Numeric                       5

Number of training examples: 99992

Labels
False    84.82%
True     15.18%
Name: fraud, dtype: object


In [25]:
# select numeric data before running AutoClassifer
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64', 'bool']
X = X.select_dtypes(include=numerics)
X_train, X_holdout, y_train, y_holdout = evalml.preprocessing.split_data(X, y, test_size=.8, random_state=0)

Because the fraud labels are binary, we will use `AutoClassifier`. When we call `.fit()`, the search for the best pipeline will begin. 

In [5]:
clf = evalml.AutoClassifier(objective=fraud_objective,
                            max_time=120,
                            max_pipelines=10)

clf.fit(X_train, y_train)

*****************************
* Beginning pipeline search *
*****************************

Optimizing for Fraud Detection. Lower score is better.

Searching up to 10 pipelines. Will stop searching for new pipelines after 120 seconds.

Possible model types: xgboost, random_forest, linear_model

Testing XGBoost w/ imputation: 100%|██████████| 10/10 [01:30<00:00,  9.05s/it]                    

✔ Optimization finished


### View rankings and select pipeline

Once the fitting process is done, we can see all of the pipelines that were searched, ranked by their score on the fraud detection objective we defined

In [6]:
clf.rankings

,id,pipeline_name,score,high_variance_cv,parameters
0,1,XGBoostPipeline,0.007631,False,"{'eta': 0.5928446182250184, 'min_child_weight'..."
1,8,RFClassificationPipeline,0.007631,False,"{'n_estimators': 609, 'max_depth': 7, 'impute_..."
2,0,RFClassificationPipeline,0.007631,False,"{'n_estimators': 569, 'max_depth': 22, 'impute..."
3,2,XGBoostPipeline,0.007631,False,"{'eta': 0.38438170729269994, 'min_child_weight..."
4,3,RFClassificationPipeline,0.007631,False,"{'n_estimators': 369, 'max_depth': 10, 'impute..."
5,4,XGBoostPipeline,0.007631,False,"{'eta': 0.5288949197529046, 'min_child_weight'..."
6,6,XGBoostPipeline,0.007631,False,"{'eta': 0.6481718720511973, 'min_child_weight'..."
7,9,XGBoostPipeline,0.007631,False,"{'eta': 0.9786183422327642, 'min_child_weight'..."
8,7,LogisticRegressionPipeline,0.007631,False,"{'penalty': 'l2', 'C': 6.239401330891865, 'imp..."
9,5,LogisticRegressionPipeline,0.007631,False,"{'penalty': 'l2', 'C': 8.444214828324364, 'imp..."


to select the best pipeline we can run

In [7]:
best_pipeline = clf.best_pipeline

### Describe pipeline

You can get more details about any pipeline. Including how it performed on other objective functions.

In [8]:
clf.describe_pipeline(1)

************************
* Pipeline Description *
************************

Pipeline Name: XGBoost w/ imputation
Model type: xgboost
Objective: Fraud Detection (lower is better)
Total training time (including CV): 4.7 seconds

Parameters
• eta: 0.5928446182250184
• min_child_weight: 8.598391737229157
• max_depth: 4
• impute_strategy: most_frequent
• percent_features: 0.6273280598181127

Cross Validation
        F1  Precision  Recall   AUC  Log Loss  Fraud Detection
0    0.264      0.152   0.264 0.500    29.296            0.008
1    0.264      0.152   0.264 0.500    29.280            0.008
2    0.264      0.152   0.264 0.500    29.296            0.008
mean 0.264      0.152   0.264 0.500    29.291            0.008
std  0.000      0.000   0.000 0.000     0.009            0.000


## Evaluate on hold out

Finally, we retrain the best pipeline on all of the training data and evaluate on the holdout

In [9]:
best_pipeline.fit(X_train, y_train)

Now, we can score the pipeline on the hold out data. 

In [10]:
best_pipeline.score(X_holdout, y_holdout, other_objectives=["auc"])

(0.007625574842707381, [0.5000368454407451])

## Why optimize for a problem-specific objective?

To demonstrate the importance of optimzing for the right objective, let's search for another pipeline using AUC, a common machine learning metric. After that, we will score the holdout data using the fraud cost objective to see how the best pipelines compare.

In [11]:
clf = evalml.AutoClassifier(objective='auc',
                            max_time=120,
                            max_pipelines=10)

clf.fit(X_train, y_train)

*****************************
* Beginning pipeline search *
*****************************

Optimizing for AUC. Greater score is better.

Searching up to 10 pipelines. Will stop searching for new pipelines after 120 seconds.

Possible model types: xgboost, random_forest, linear_model

Testing Random Forest w/ imputation: 100%|██████████| 10/10 [01:07<00:00,  6.72s/it]              

✔ Optimization finished


like before, we can look at the rankings and pick the best pipeline

In [15]:
clf.rankings

,id,pipeline_name,score,high_variance_cv,parameters
0,6,XGBoostPipeline,0.833904,False,"{'eta': 0.38438170729269994, 'min_child_weight..."
1,1,RFClassificationPipeline,0.832827,False,"{'n_estimators': 569, 'max_depth': 22, 'impute..."
2,4,XGBoostPipeline,0.832510,False,"{'eta': 0.5928446182250184, 'min_child_weight'..."
3,9,RFClassificationPipeline,0.832510,False,"{'n_estimators': 609, 'max_depth': 7, 'impute_..."
4,2,RFClassificationPipeline,0.832333,False,"{'n_estimators': 369, 'max_depth': 10, 'impute..."
5,0,LogisticRegressionPipeline,0.658597,False,"{'penalty': 'l2', 'C': 8.444214828324364, 'imp..."
6,3,LogisticRegressionPipeline,0.658597,False,"{'penalty': 'l2', 'C': 6.239401330891865, 'imp..."
7,7,LogisticRegressionPipeline,0.658597,False,"{'penalty': 'l2', 'C': 8.123565600467177, 'imp..."
8,8,LogisticRegressionPipeline,0.658597,False,"{'penalty': 'l2', 'C': 8.362426847738403, 'imp..."
9,5,LogisticRegressionPipeline,0.656621,False,"{'penalty': 'l2', 'C': 0.5765626434012575, 'im..."


In [18]:
clf.describe_pipeline(1)

************************
* Pipeline Description *
************************

Pipeline Name: Random Forest w/ imputation
Model type: random_forest
Objective: AUC (greater is better)
Total training time (including CV): 23.6 seconds

Parameters
• n_estimators: 569
• max_depth: 22
• impute_strategy: most_frequent
• percent_features: 0.8593661614465293

Cross Validation
        F1  Precision  Recall   AUC  Log Loss
0    0.814      1.000   0.814 0.843     1.648
1    0.812      1.000   0.812 0.842     1.658
2    0.799      0.997   0.799 0.833     1.762
mean 0.808      0.999   0.808 0.839     1.689
std  0.008      0.002   0.008 0.006     0.063


In [16]:
best_pipeline_auc = clf.best_pipeline

# train on the full training data
best_pipeline_auc.fit(X_train, y_train)

In [17]:
# get the fraud score on holdout data
score, [fraud_score] = best_pipeline_auc.score(X_holdout, y_holdout, other_objectives=[fraud_objective])
fraud_score

0.03433982479902143

We can see that in the model optimized for AUC we lose ~3% of transaction amount to fraud compared to under 1% when we optmize for it specifically. 

This is because the AUC doesn't take into account that before labeling a transaction as fraud or not, we need to take into account the cost/benifiet of that decision taking into acount things like the amount of the transaction. 